In [1]:
import pandas as pd
import numpy as np 


df = pd.read_pickle('C:/Users/Rafael/OneDrive/Documentos/TCC Rafaela/Dados/Pickle/cancer_mama_regiao_sudeste_sem_nulos.pickle')


In [2]:
# df['AP_CNSPCN'].head()

pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN', 'AP_PRIPAL'], 
                             values=['AP_DTINIC', 'AQ_ESTADI'], 
                             aggfunc=['min', 'max'])
pivot_table.head()

min                   max          
                            AP_DTINIC AQ_ESTADI   AP_DTINIC AQ_ESTADI
AP_CNSPCN       AP_PRIPAL                                            
8980500671835 8 304050121  01/03/2009         2  01/03/2009         2
{{ü}{{üÇÇ{{{| 304050121  12/04/2010         3  12/04/2010         3
{}{|}üäââü{{|â 304050040  12/03/2010         1  23/12/2010         1
                304050130  07/10/2009         1  12/01/2010         1
{}{ä~äÇ~~||{{{~ 304050067  15/02/2011         3  15/02/2011         3

In [3]:
#Calcular a diferença do estadiamento entre os períodos

pivot_table['dif_estadiamento'] = pivot_table[('max','AQ_ESTADI')]-pivot_table[('min','AQ_ESTADI')]

#Contar a qauntidade de cada valor
pivot_table['dif_estadiamento'].value_counts()

dif_estadiamento
0    624818
1     25374
2     14699
3      6864
4      3209
Name: count, dtype: int64

In [4]:
pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN'], 
                             values=['AP_DTINIC', 'AQ_ESTADI'], 
                             aggfunc=['min', 'max'])

#Cálculo da Diferença entre o Máximo e Mínimo do Estadiamento
pivot_table['dif_estadiamento'] = pivot_table[('max','AQ_ESTADI')]-pivot_table[('min','AQ_ESTADI')]
pivot_table['dif_estadiamento'].value_counts()

dif_estadiamento
0    292262
1     45320
2     23542
3     10136
4      4705
Name: count, dtype: int64

In [5]:
# Renomear as colunas da tabela dinâmica
pivot_table = pivot_table.set_axis(['inicio','estadiamento_inicio','fim','estadiamento_fim','dif_estadiamento'], axis=1)
pivot_table.head()

,inicio,estadiamento_inicio,fim,estadiamento_fim,dif_estadiamento
AP_CNSPCN,,,,,
8980500671835 8,01/03/2009,2,01/03/2009,2,0
{{ü}{{üÇÇ{{{|,12/04/2010,3,12/04/2010,3,0
{}{|}üäââü{{|â,07/10/2009,1,23/12/2010,1,0
{}{ä~äÇ~~||{{{~,15/02/2011,3,15/02/2011,3,0
{}|{|éäâé|ü{{{{,01/11/2010,4,01/11/2010,4,0


In [6]:
# Merge da tabela pivotada e do dataset
df = pd.merge(df, pivot_table, on=['AP_CNSPCN'], how='left')
df.head()

,AP_PRIPAL,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,esquema_terapeutico,Numero_de_Tratamentos_Anteriores,Numero_de_Diagnosticos_Diferentes,Tem_Causas_Associadas,Tem_Causas_Secundaria,inicio,estadiamento_inicio,fim,estadiamento_fim,dif_estadiamento
0,304050121,ES,âäâ{Ç{{||ÇÇäüéé,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,TMX,2,1,1,1,01/01/2008,2,01/07/2008,2,0
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,TAMOXIFENO,3,1,1,1,01/01/2008,1,29/01/2013,4,3
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,TAMOXIFENO,3,1,1,1,01/01/2008,2,01/10/2008,2,0
3,304020346,ES,âäâ{{|{}â|éâ|},47,F,3,ES,Brasil,29/01/2008,31/03/2008,...,TAMOXIFENO,0,1,1,1,11/04/2008,4,29/01/2008,4,0
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,TAMOXIFENO,2,1,1,1,01/01/2008,2,03/01/2012,2,0


In [7]:
# Pivotar tabela
pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN', 'AP_PRIPAL'], 
                             values=['AP_DTINIC', 'AQ_ESTADI'], 
                             aggfunc=['min', 'max'])

# Calcular o total de tratamento realizados pelo paciente
total_tratamentos = pivot_table.reset_index().groupby('AP_CNSPCN')['AP_PRIPAL'].count()

#Fazer o merge dessa coluna de total de tratamentos ao dataset.
df = pd.merge(df, total_tratamentos, on=['AP_CNSPCN'], how='left')
df.head()


,AP_PRIPAL_x,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,Numero_de_Tratamentos_Anteriores,Numero_de_Diagnosticos_Diferentes,Tem_Causas_Associadas,Tem_Causas_Secundaria,inicio,estadiamento_inicio,fim,estadiamento_fim,dif_estadiamento,AP_PRIPAL_y
0,304050121,ES,âäâ{Ç{{||ÇÇäüéé,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,2,1,1,1,01/01/2008,2,01/07/2008,2,0,1
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,3,1,1,1,01/01/2008,1,29/01/2013,4,3,4
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,3,1,1,1,01/01/2008,2,01/10/2008,2,0,1
3,304020346,ES,âäâ{{|{}â|éâ|},47,F,3,ES,Brasil,29/01/2008,31/03/2008,...,0,1,1,1,11/04/2008,4,29/01/2008,4,0,2
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,2,1,1,1,01/01/2008,2,03/01/2012,2,0,2


In [8]:
# Renomeando as colunas
df = df.rename(columns={'AP_PRIPAL_x': 'AP_PRIPAL'})
df = df.rename(columns={'AP_PRIPAL_y': 'total_tratamentos'})
df.head()

,AP_PRIPAL,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,Numero_de_Tratamentos_Anteriores,Numero_de_Diagnosticos_Diferentes,Tem_Causas_Associadas,Tem_Causas_Secundaria,inicio,estadiamento_inicio,fim,estadiamento_fim,dif_estadiamento,total_tratamentos
0,304050121,ES,âäâ{Ç{{||ÇÇäüéé,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,2,1,1,1,01/01/2008,2,01/07/2008,2,0,1
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,3,1,1,1,01/01/2008,1,29/01/2013,4,3,4
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,3,1,1,1,01/01/2008,2,01/10/2008,2,0,1
3,304020346,ES,âäâ{{|{}â|éâ|},47,F,3,ES,Brasil,29/01/2008,31/03/2008,...,0,1,1,1,11/04/2008,4,29/01/2008,4,0,2
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,2,1,1,1,01/01/2008,2,03/01/2012,2,0,2


In [9]:
#Pivot da tabela
pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN', 'AP_PRIPAL'], 
                             values=['AP_DTINIC'], 
                             aggfunc=['min', 'max'])
procedimento_inicial = pivot_table.reset_index().groupby('AP_CNSPCN')['AP_PRIPAL'].first()

# Obtendo o Primeiro Procedimento por Paciente
df = pd.merge(df, procedimento_inicial, on=['AP_CNSPCN'], how='left')
df.head()

,AP_PRIPAL_x,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,Numero_de_Diagnosticos_Diferentes,Tem_Causas_Associadas,Tem_Causas_Secundaria,inicio,estadiamento_inicio,fim,estadiamento_fim,dif_estadiamento,total_tratamentos,AP_PRIPAL_y
0,304050121,ES,âäâ{Ç{{||ÇÇäüéé,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,2,01/07/2008,2,0,1,304050121
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,1,29/01/2013,4,3,4,304020133
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,2,01/10/2008,2,0,1,304050121
3,304020346,ES,âäâ{{|{}â|éâ|},47,F,3,ES,Brasil,29/01/2008,31/03/2008,...,1,1,1,11/04/2008,4,29/01/2008,4,0,2,304020338
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,2,03/01/2012,2,0,2,304050105


In [10]:
#Renomear colunas 
df = df.rename(columns={'AP_PRIPAL_x': 'AP_PRIPAL'})
df = df.rename(columns={'AP_PRIPAL_y': 'procedimento_inicial'})
df.head()

,AP_PRIPAL,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,Numero_de_Diagnosticos_Diferentes,Tem_Causas_Associadas,Tem_Causas_Secundaria,inicio,estadiamento_inicio,fim,estadiamento_fim,dif_estadiamento,total_tratamentos,procedimento_inicial
0,304050121,ES,âäâ{Ç{{||ÇÇäüéé,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,2,01/07/2008,2,0,1,304050121
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,1,29/01/2013,4,3,4,304020133
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,2,01/10/2008,2,0,1,304050121
3,304020346,ES,âäâ{{|{}â|éâ|},47,F,3,ES,Brasil,29/01/2008,31/03/2008,...,1,1,1,11/04/2008,4,29/01/2008,4,0,2,304020338
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,1,01/01/2008,2,03/01/2012,2,0,2,304050105


In [11]:
# Criação da coluna Piora
df['piora'] = df['dif_estadiamento'].apply(lambda x: 1 if x > 0 else 0)
df['piora'].value_counts()

#Indicador de óbito 
df['obito'] = df['AP_OBITO'].apply(lambda x: 1 if x > 0 else 0)
df['obito'].value_counts()

#Indicador de Alta 
df['alta'] = df['AP_ALTA'].apply(lambda x: 1 if x > 0 else 0)
df['alta'].value_counts()

#Indicador de Melhora
df['melhora'] = df['AP_ENCERR'].apply(lambda x: 1 if x > 0 else 0)
df['melhora'].value_counts()

melhora
0    18228112
1      194914
Name: count, dtype: int64

In [12]:
#Pivot da tabela por CNS
pivot_table = pd.pivot_table(df,
                             index='AP_CNSPCN',
                             values=['obito','piora','melhora','alta'],
                             aggfunc='max'
                             )
pivot_table.head()

,alta,melhora,obito,piora
AP_CNSPCN,,,,
8980500671835 8,0,0,0,0
{{ü}{{üÇÇ{{{|,0,0,0,0
{}{|}üäââü{{|â,0,0,0,0
{}{ä~äÇ~~||{{{~,0,0,0,0
{}|{|éäâé|ü{{{{,0,0,0,0


In [13]:
# Verificar desfecho do Paciente
def verifica_desfecho(obito, piora, melhora, alta):
    if(obito > 0 ):
        return 'Óbito'
    elif(piora > 0 ):
        return 'Piora'
    elif(melhora > 0):
        return 'Melhora'
    elif(alta > 0):
        return 'Alta'
    else:
        return 'SemDesfecho'

#Criar coluna com esse desfecho
df['desfecho'] = np.vectorize(verifica_desfecho)(df['obito'], df['piora'], df['melhora'], df['alta'])
df['desfecho'].value_counts()

desfecho
SemDesfecho    11763907
Piora           6103810
Alta             392971
Melhora          138621
Óbito             23717
Name: count, dtype: int64

In [14]:
df.drop(columns=['AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA'], inplace=True)

df.head()

,AP_PRIPAL,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,fim,estadiamento_fim,dif_estadiamento,total_tratamentos,procedimento_inicial,piora,obito,alta,melhora,desfecho
0,304050121,ES,âäâ{Ç{{||ÇÇäüéé,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,01/07/2008,2,0,1,304050121,0,0,0,0,SemDesfecho
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,29/01/2013,4,3,4,304020133,1,0,0,0,Piora
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,01/10/2008,2,0,1,304050121,0,0,0,0,SemDesfecho
3,304020346,ES,âäâ{{|{}â|éâ|},47,F,3,ES,Brasil,29/01/2008,31/03/2008,...,29/01/2008,4,0,2,304020338,0,0,0,0,SemDesfecho
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,03/01/2012,2,0,2,304050105,0,0,0,0,SemDesfecho


In [15]:
pivot_table['desfecho'] = np.vectorize(verifica_desfecho)(pivot_table['obito'], pivot_table['piora'], pivot_table['melhora'], pivot_table['alta'])
pivot_table['desfecho'].value_counts()
df = pd.merge(df, pivot_table, on='AP_CNSPCN', how='left')

In [16]:
# Renomear a coluna
df = df.rename(columns={'desfecho_y': 'desfecho'})
df.head()

,AP_PRIPAL,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,piora_x,obito_x,alta_x,melhora_x,desfecho_x,alta_y,melhora_y,obito_y,piora_y,desfecho
0,304050121,ES,âäâ{Ç{{||ÇÇäüéé,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,0,0,0,0,SemDesfecho,0,0,0,0,SemDesfecho
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,0,0,0,Piora,1,0,0,1,Piora
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,0,0,0,0,SemDesfecho,1,0,0,0,Alta
3,304020346,ES,âäâ{{|{}â|éâ|},47,F,3,ES,Brasil,29/01/2008,31/03/2008,...,0,0,0,0,SemDesfecho,0,0,0,0,SemDesfecho
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,0,0,0,0,SemDesfecho,1,0,0,0,Alta


In [17]:
# Remover linhas sem desfecho
df = df.loc[df['desfecho'] != 'SemDesfecho',:]

In [18]:
# Excluindo as colunas 'AQ_ESQU_P1' e 'AQ_ESQU_P2'
df.drop(columns=['piora_x',	'obito_x',	'alta_x',	'melhora_x',	'desfecho_x',	'alta_y',	'melhora_y',	'obito_y',	'piora_y'], inplace=True)

df.head()

,AP_PRIPAL,AP_UFMUN,AP_CNSPCN,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_DTINIC,AP_DTFIM,...,Tem_Causas_Associadas,Tem_Causas_Secundaria,inicio,estadiamento_inicio,fim,estadiamento_fim,dif_estadiamento,total_tratamentos,procedimento_inicial,desfecho
1,304050040,ES,âäâ{Ç{{{{üé}Çüü,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,01/01/2008,1,29/01/2013,4,3,4,304020133,Piora
2,304050121,ES,}{{éüâ}â{Ç{{{~,61,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,01/01/2008,2,01/10/2008,2,0,1,304050121,Alta
4,304050105,ES,âäâ{{{|{~ü}~ü,71,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,01/01/2008,2,03/01/2012,2,0,2,304050105,Alta
5,304050121,ES,âäâ{{{|{~é{Ç~â,51,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,01/01/2008,2,01/10/2010,2,0,1,304050121,Alta
6,304050113,ES,âäâ{Ç{{|}{ääÇ,40,F,99,ES,Brasil,01/01/2008,31/03/2008,...,1,1,01/01/2008,3,01/10/2010,3,0,1,304050113,Alta


In [19]:
import pickle

dados_serializados = pickle.dumps(df)
with open('C:/Users/Rafael/OneDrive/Documentos/TCC Rafaela/Dados/Pickle/cancer_mama_regiao_sudeste_com_desfecho.pickle', 'wb') as arquivo:
    arquivo.write(dados_serializados)